In [13]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import backtrader as bt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pyfolio as pf
import quantstats
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
import sys
import time
import sqlite3
from scipy.stats import rankdata
from scipy.stats import stats
from scipy.optimize import minimize

미국 상장 ETF들의 일간 가격 정보를 다운로드 받을 것입니다. ETF Ticker는 나스닥의 ftp(ftp.nasdaqtrader.com/SymbolDirectory)에서 가져왔고, 가공했습니다. 상장된 지 가장 오래된 ETF인 SPY(SPDR S&P 500 ETF Trust)가 1993년 1월 22일 상장이므로, 그 시점부터 시작하겠습니다.

In [2]:
nasdaqlisted = pd.read_excel('nasdaqlisted.xlsx', sheet_name='nasdaqlistedetf', encoding='utf-8')
otherlisted = pd.read_excel('otherlisted.xlsx', sheet_name='otherlistedetf', encoding='utf-8')

In [3]:
nasdaqlisted.head()

,Symbol,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AAXJ,iShares MSCI All Country Asia ex Japan Index Fund,G,N,N,100,Y,N
1,ACWI,iShares MSCI ACWI Index Fund,G,N,N,100,Y,N
2,ACWX,iShares MSCI ACWI ex US Index Fund,G,N,N,100,Y,N
3,ADRE,Invesco BLDRS Emerging Markets 50 ADR Index Fund,G,N,N,100,Y,N
4,AGZD,WisdomTree Interest Rate Hedged U.S. Aggregate...,G,N,N,100,Y,N


In [4]:
len(nasdaqlisted)

407

In [5]:
otherlisted.head()

,ACT Symbol,Security Name,Exchange,CQS Symbol,ETF,Round Lot Size,Test Issue,NASDAQ Symbol
0,AAA,Listed Funds Trust AAF First Priority CLO Bond...,P,AAA,Y,100,N,AAA
1,AAAU,Goldman Sachs Physical Gold ETF Shares,P,AAAU,Y,100,N,AAAU
2,AADR,AdvisorShares Dorsey Wright ADR ETF,P,AADR,Y,100,N,AADR
3,ABEQ,Absolute Core Strategy ETF,P,ABEQ,Y,100,N,ABEQ
4,ACES,ALPS Clean Energy ETF,Z,ACES,Y,100,N,ACES


In [6]:
len(otherlisted)

1940

In [10]:
ticker_list = list(nasdaqlisted['Symbol']) + list(otherlisted['NASDAQ Symbol'])
ticker_list[0:5]

['AAXJ', 'ACWI', 'ACWX', 'ADRE', 'AGZD']

In [11]:
len(ticker_list)

2347

Yahoo Finance에서 OHLCV 데이터를 가져옵니다. USListedETFPrice.db 이름의 sqlite 파일에 저장되고, ticker 이름으로 된 테이블에 데이터가 들어갑니다.

In [12]:
def OHLCV_Download(ticker, start, end):
    df = web.DataReader(ticker, 'yahoo', start, end)
    con = sqlite3.connect('USListedETFPrice.db')
    df.to_sql(ticker, con, if_exists='append')
    con.close()

차단 방지를 위해 50개 ticker 호출마다 10초씩 멈춥니다.

In [19]:
i = 1
for ticker in ticker_list:
    try:
        print((i, ticker))

        OHLCV_Download(ticker, '1993-01-22', '2021-02-19')

        i += 1

        if i % 50 == 0:
            time.sleep(10)
        
    except:
        print((i, ticker, 'Error Occurred'))
        OHLCV_Download(ticker, '1993-01-22', '2021-02-19')

(1, 'AAXJ')
(2, 'ACWI')
(3, 'ACWX')
(4, 'ADRE')
(5, 'AGZD')
(6, 'AIA')
(7, 'AIQ')
(8, 'AIRR')
(9, 'ALTY')
(10, 'AMCA')
(11, 'ANGL')
(12, 'ASET')
(13, 'BBH')
(14, 'BFIT')
(15, 'BGRN')
(16, 'BIB')
(17, 'BICK')
(18, 'BIS')
(19, 'BJK')
(20, 'BLCN')
(21, 'BND')
(22, 'BNDW')
(23, 'BNDX')
(24, 'BOTZ')
(25, 'BSAE')
(26, 'BSBE')
(27, 'BSCE')
(28, 'BSCL')
(29, 'BSCM')
(30, 'BSCN')
(31, 'BSCO')
(32, 'BSCP')
(33, 'BSCQ')
(34, 'BSCR')
(35, 'BSCS')
(36, 'BSCT')
(37, 'BSCU')
(38, 'BSDE')
(39, 'BSJL')
(40, 'BSJM')
(41, 'BSJN')
(42, 'BSJO')
(43, 'BSJP')
(44, 'BSJQ')
(45, 'BSJR')
(46, 'BSJS')
(47, 'BSML')
(48, 'BSMM')
(49, 'BSMN')
(50, 'BSMO')
(51, 'BSMP')
(52, 'BSMQ')
(53, 'BSMR')
(54, 'BSMS')
(55, 'BSMT')
(56, 'BSMU')
(57, 'BTEC')
(58, 'BUG')
(59, 'CACG')
(60, 'CARZ')
(61, 'CATH')
(62, 'CDC')
(63, 'CDL')
(64, 'CEFA')
(65, 'CEY')
(66, 'CEZ')
(67, 'CFA')
(68, 'CFO')
(69, 'CHB')
(70, 'CHNA')
(71, 'CIBR')
(72, 'CID')
(73, 'CIL')
(74, 'CIZ')
(75, 'CLOU')
(76, 'CLRG')
(77, 'CNCR')
(78, 'COMT')
(79, 'CSA')
(

(604, 'COM')
(605, 'COMB')
(606, 'COPX')
(607, 'CORN')
(608, 'CORP')
(609, 'COWZ')
(610, 'CPER')
(611, 'CPI')
(612, 'CQQQ')
(613, 'CRAK')
(614, 'CRBN')
(615, 'CSD')
(616, 'CSM')
(617, 'CURE')
(618, 'CUT')
(619, 'CVY')
(620, 'CWB')
(621, 'CWEB')
(622, 'CWI')
(623, 'CWS')
(624, 'CYB')
(625, 'CZA')
(626, 'DALT')
(627, 'DAUG')
(628, 'DBA')
(629, 'DBAW')
(630, 'DBB')
(631, 'DBC')
(632, 'DBE')
(633, 'DBEF')
(634, 'DBEH')
(635, 'DBEM')
(636, 'DBEU')
(637, 'DBEZ')
(638, 'DBGR')
(639, 'DBJA')
(640, 'DBJP')
(641, 'DBLV')
(642, 'DBMF')
(643, 'DBO')
(644, 'DBOC')
(645, 'DBP')
(646, 'DBS')
(647, 'DBV')
(648, 'DDEC')
(649, 'DDG')
(650, 'DDLS')
(651, 'DDM')
(652, 'DDWM')
(653, 'DECZ')
(654, 'DEED')
(655, 'DEEF')
(656, 'DEEP')
(657, 'DEF')
(658, 'DEFA')
(659, 'DEFN')
(660, 'DEM')
(661, 'DES')
(662, 'DEUS')
(663, 'DEW')
(664, 'DFAE')
(665, 'DFAI')
(666, 'DFAU')
(667, 'DFE')
(668, 'DFEB')
(669, 'DFEN')
(670, 'DFHY')
(671, 'DFJ')
(672, 'DFND')
(673, 'DFNV')
(674, 'DGL')
(675, 'DGRO')
(676, 'DGS')
(677, '

(1189, 'IBDP')
(1190, 'IBDQ')
(1191, 'IBDR')
(1192, 'IBDS')
(1193, 'IBDT')
(1194, 'IBDU')
(1195, 'IBDV')
(1196, 'IBHA')
(1197, 'IBHB')
(1198, 'IBHC')
(1199, 'IBHD')
(1200, 'IBHE')
(1201, 'IBHF')
(1202, 'IBMJ')
(1203, 'IBMK')
(1204, 'IBML')
(1205, 'IBMM')
(1206, 'IBMN')
(1207, 'IBMO')
(1208, 'IBMP')
(1209, 'IBMQ')
(1210, 'IBND')
(1211, 'IBUY')
(1212, 'ICF')
(1213, 'ICOL')
(1214, 'ICOW')
(1215, 'ICSH')
(1216, 'ICVT')
(1217, 'IDEV')
(1218, 'IDHD')
(1219, 'IDHQ')
(1220, 'IDIV')
(1221, 'IDLV')
(1222, 'IDMO')
(1223, 'IDNA')
(1224, 'IDOG')
(1225, 'IDRV')
(1226, 'IDU')
(1227, 'IDV')
(1228, 'IDX')
(1229, 'IECS')
(1230, 'IEDI')
(1231, 'IEFA')
(1232, 'IEFN')
(1233, 'IEHS')
(1234, 'IEIH')
(1235, 'IEME')
(1236, 'IEMG')
(1237, 'IEO')
(1238, 'IETC')
(1239, 'IEUR')
(1240, 'IEV')
(1241, 'IEZ')
(1242, 'IFRA')
(1243, 'IG')
(1244, 'IGBH')
(1245, 'IGE')
(1246, 'IGEB')
(1247, 'IGHG')
(1248, 'IGLB')
(1249, 'IGM')
(1250, 'IGN')
(1251, 'IGRO')
(1252, 'IGV')
(1253, 'IHAK')
(1254, 'IHDG')
(1255, 'IHE')
(1256, 'I

(1747, 'QLS')
(1748, 'QLTA')
(1749, 'QLV')
(1750, 'QLVD')
(1751, 'QLVE')
(1752, 'QMJ')
(1753, 'QMN')
(1754, 'QMOM')
(1755, 'QPT')
(1756, 'QPX')
(1757, 'QQH')
(1758, 'QQQE')
(1759, 'QRFT')
(1760, 'QSY')
(1761, 'QTUM')
(1762, 'QUAL')
(1763, 'QULL')
(1764, 'QUS')
(1765, 'QVAL')
(1766, 'QWLD')
(1767, 'RAAX')
(1768, 'RAFE')
(1769, 'RALS')
(1770, 'RAVI')
(1771, 'RAYC')
(1772, 'RBIN')
(1773, 'RBND')
(1774, 'RBUS')
(1775, 'RCD')
(1776, 'RDFI')
(1777, 'RDIV')
(1778, 'RDOG')
(1779, 'REC')
(1780, 'RECS')
(1781, 'REET')
(1782, 'REGL')
(1783, 'REK')
(1784, 'REM')
(1785, 'REMX')
(1786, 'RESD')
(1787, 'RESE')
(1788, 'RESP')
(1789, 'RETL')
(1790, 'REVS')
(1791, 'REW')
(1792, 'REZ')
(1793, 'RFCI')
(1794, 'RFDA')
(1795, 'RFFC')
(1796, 'RFG')
(1797, 'RFUN')
(1798, 'RFV')
(1799, 'RGI')
(1800, 'RHS')
(1801, 'RIGS')
(1802, 'RINF')
(1803, 'RISN')
(1804, 'RLY')
(1805, 'ROAM')
(1806, 'ROBO')
(1807, 'RODE')
(1808, 'RODM')
(1809, 'ROKT')
(1810, 'ROM')
(1811, 'ROMO')
(1812, 'ROOF')
(1813, 'RORO')
(1814, 'ROSC')
(

(2306, 'XMLV')
(2307, 'XMMO')
(2308, 'XMPT')
(2309, 'XMVM')
(2310, 'XNTK')
(2311, 'XOP')
(2312, 'XOUT')
(2313, 'XPH')
(2314, 'XPP')
(2315, 'XRLV')
(2316, 'XRT')
(2317, 'XSD')
(2318, 'XSHD')
(2319, 'XSHQ')
(2320, 'XSLV')
(2321, 'XSMO')
(2322, 'XSOE')
(2323, 'XSVM')
(2324, 'XSW')
(2325, 'XTL')
(2326, 'XTN')
(2327, 'XVV')
(2328, 'XWEB')
(2329, 'XYLD')
(2330, 'XYLG')
(2331, 'YANG')
(2332, 'YCL')
(2333, 'YCS')
(2334, 'YDEC')
(2335, 'YINN')
(2336, 'YLD')
(2337, 'YOLO')
(2338, 'YXI')
(2339, 'YYY')
(2340, 'ZCAN')
(2341, 'ZDEU')
(2342, 'ZGBR')
(2343, 'ZHOK')
(2344, 'ZIG')
(2345, 'ZJPN')
(2346, 'ZROZ')
(2347, 'ZSL')


다운받은 데이터 중 가장 긴 기간을 가지는 SPY ETF 데이터가 잘 들어왔습니다.

In [20]:
start = '1993-01-22'
end = '2021-02-19'

con = sqlite3.connect('USListedETFPrice.db')
read_SPY = pd.read_sql("select * from 'SPY' ", con, index_col='Date')
read_SPY

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
1993-01-29 00:00:00,43.968750,43.750000,43.968750,43.937500,1003200.0,25.968958
1993-02-01 00:00:00,44.250000,43.968750,43.968750,44.250000,480500.0,26.153660
1993-02-02 00:00:00,44.375000,44.125000,44.218750,44.343750,201300.0,26.209057
1993-02-03 00:00:00,44.843750,44.375000,44.406250,44.812500,529400.0,26.486113
1993-02-04 00:00:00,45.093750,44.468750,44.968750,45.000000,531500.0,26.596937
...,...,...,...,...,...,...
2021-02-12 00:00:00,392.899994,389.769989,389.850006,392.640015,50593300.0,392.640015
2021-02-16 00:00:00,394.170013,391.529999,393.959991,392.299988,50972400.0,392.299988
2021-02-17 00:00:00,392.660004,389.329987,390.420013,392.390015,52290600.0,392.390015


In [21]:
con.close()